In [1]:
# import libraries
import os
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

OPENAI_API_KEY = st.secrets['openai_api_key'] # add your OpenAI API Key
DOC_PATH = "files/whole_bible/NIV-Bible-PDF.pdf"
CHROMA_PATH = "bible_chromadb" 

# ----- Data Indexing Process -----

# load your pdf doc
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

# split the doc into smaller chunks i.e. chunk_size=500
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(pages)

# get OpenAI Embedding model
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

c:\Users\lets_\Documents\bible-rag\bibleenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [6]:
from pinecone import Pinecone, ServerlessSpec
# create new pinecone index
pc = Pinecone(api_key=st.secrets["pinecone_api_key"])
pc.create_index(
    name="full-bible-index",
    dimension=1536, # Replace with your model dimensions
    metric="euclidean", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [7]:
from pinecone import Pinecone
pc = Pinecone(api_key=os.environ.get("pinecone_api_key"))

from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnablePassthrough

index_name = "full-bible-index"

# connect to index
index = pc.Index(index_name)

# view index stats
index.describe_index_stats()

pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)


In [8]:
pinecone.similarity_search("What is the pentacost?")[:3]

[Document(page_content='2Suddenly a sound like the blowing of a \nviolent wind came from heaven and filled the whole house where they were \nsitting. \n \n3They saw what seemed to be tongues \nof fire that separated and came to rest \non each of them.  \n4All of them were filled with the Holy \nSpirit and began to speak in other tongues as the Spirit enabled them. \n \n5Now there were staying in Jerusalem \nGod-fearing Jews from every nation \nunder heaven.  \n6When they heard this sound, a crowd \ncame together in bewilderment,', metadata={'page': 1512.0, 'source': 'files/whole_bible/NIV-Bible-PDF.pdf'}),
 Document(page_content='19On the evening of that first day of the \nweek, when the disciples were together, \nwith the doors locked for fear of the Jews, Jesus came and stood among \nthem and said, "Peace be with you!" \n \n20After he said this, he showed them his \nhands and side. The disciples were overjoyed when they  saw the Lord. \n \n21Again Jesus said, "Peace be with you! \nAs

In [14]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


template = """
You are a friendly chatbot who has access to the Christian bible. 
Several relevant bible passages will be given to you in the context. 
Answer the question based on the context. 
If you can't answer the question, reply "I don't know".
If you are going to answer the question, mention the bible verse that you are talking about.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# load model from open AI
model = ChatOpenAI(model_name = "gpt-3.5-turbo-0125",
                    openai_api_key = st.secrets['openai_api_key'],
                    temperature = 0.2, 
                    streaming=True)


chain = (
    {"context": pinecone.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | OpenAIFunctionsAgentOutputParser()
)


In [15]:
chain.invoke("What does Jesus say about suffering?")

AgentFinish(return_values={'output': 'In the context provided, Jesus talks about suffering in Mark 9:12-13, where he mentions that the Son of Man must suffer much and be rejected. He also mentions that in the same way, the Son of Man is going to suffer at the hands of others. This shows that Jesus acknowledges and foretells the suffering he will endure, emphasizing the importance of enduring suffering as part of his mission.'}, log='In the context provided, Jesus talks about suffering in Mark 9:12-13, where he mentions that the Son of Man must suffer much and be rejected. He also mentions that in the same way, the Son of Man is going to suffer at the hands of others. This shows that Jesus acknowledges and foretells the suffering he will endure, emphasizing the importance of enduring suffering as part of his mission.')